In [1]:
import illustris_python as il
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
#%matplotlib widget
basePath ='./sims.TNG/TNG50-1/output'
from tqdm import tqdm
import gc

fields=['SubhaloPos','SubhaloMass']



In [2]:
class Cell:
    def __init__(self):
        self.Coordinate = [0, 0]
        self.Mass = 0

class Cloud:
    def __init__(self, i_range, j_range):
        self.cells = []
        for i in tqdm(range(i_range[0], i_range[1], 1)):
            for j in range(j_range[0], j_range[1], 1):
                cell = Cell()
                cell.Coordinate = [i+ 0.5, j + 0.5]
                self.cells.append(cell)
        self.i_range = i_range
        self.j_range = j_range
        print(len(self.cells))

    def assign_mass(self, coordinates: np.ndarray, masses: np.ndarray):
        j_min, j_max = self.j_range
        i_min, i_max = self.i_range
        j_bars = int((j_max - j_min))
        for i in range(coordinates.shape[0]):
            coord = coordinates[i]
            mass = masses[i]
            j_index = int((coord[2] - j_min))
            i_index = int((coord[1] - i_min))
            if j_index >= j_bars or i_index >= j_bars:
                continue
            index = i_index * j_bars + j_index
            self.cells[index].Mass += mass

    def visualize(self, savePath):
        plt.figure(figsize=(20, 20), dpi=400)
        plt.style.use("dark_background")
        i_range, j_range = self.i_range, self.j_range
        plt.hist2d([cell.Coordinate[0] for cell in self.cells], [cell.Coordinate[1] for cell in self.cells], norm=mpl.colors.LogNorm(), cmin=0.0001, bins=[(i_range[1] - i_range[0]), (j_range[1] - j_range[0])], weights=[cell.Mass for cell in self.cells])
        plt.xlabel('y [ckpc/h]')
        plt.ylabel('z [ckpc/h]')
        plt.xlim(self.i_range[0], self.i_range[1])
        plt.ylim(self.j_range[0], self.j_range[1])
        plt.axes().get_xaxis().set_visible(False)
        plt.axes().get_yaxis().set_visible(False)
        plt.axis('equal')
        plt.savefig(savePath)
        plt.close('all')

    def reset(self):
        for cell in self.cells:
            cell.Coordinate = [0, 0]
            cell.Mass = 0

            
    def rearrange_coordinates(self, new_i_range, new_j_range):
        i_min, i_max = self.i_range
        j_min, j_max = self.j_range
        i_new_min, i_new_max = new_i_range
        j_new_min, j_new_max = new_j_range
        
        i_scale = (i_new_max - i_new_min) / (i_max - i_min)
        j_scale = (j_new_max - j_new_min) / (j_max - j_min)
        
        for cell in self.cells:
            i, j = cell.Coordinate
            new_i = round((i - i_min) * i_scale + i_new_min)+0.5
            new_j = round((j - j_min) * j_scale + j_new_min)+0.5
            cell.Coordinate = [new_i, new_i]
            cell.Mass=0
            
        self.i_range = new_i_range
        self.j_range = new_j_range
            
            
def Binary_DataIter(Data_Set1,Data_Set2):
    for i in range(0,min(len(Data_Set1),len(Data_Set2))):
        yield Data_Set1[i],Data_Set2[i]

In [4]:
tree_fields=['SubfindID','SnapNum']
subfindid_snapnum_tree=il.sublink.loadTree(basePath, 99, 329508, fields=tree_fields, onlyMPB=True)
subfind_id=subfindid_snapnum_tree['SubfindID']
snapshot_num=subfindid_snapnum_tree['SnapNum']
del subfindid_snapnum_tree
gc.collect()

0

In [4]:
for current_subfind_id,current_snapshot_num in Binary_DataIter(subfind_id,snapshot_num):
    if current_snapshot_num < 40: break
    if current_snapshot_num > 69:continuez
        
    print(current_subfind_id, current_snapshot_num)

215980 69
214057 68
211991 67
209820 66
205211 65
184704 64
171321 63
170176 62
163331 61
162126 60
158822 59
156135 58
157140 57
154894 56
152796 55
144633 54
145765 53
140683 52
156816 51
153494 50
151763 49
145660 48
139042 47
130895 46
121961 45
120173 44
118400 43
119619 42
107743 41
98181 40


In [3]:
current_subfind_id,current_snapshot_num=98181, 40
subhalo_data=il.groupcat.loadSubhalos(basePath, current_snapshot_num, fields=fields)
Positions=subhalo_data['SubhaloPos']
Mass=subhalo_data['SubhaloMass']
GalaxyPos=Positions[current_subfind_id]
    
ymin=GalaxyPos[1]-2600
ymax=GalaxyPos[1]+2600
    
zmin=GalaxyPos[2]-2600
zmax=GalaxyPos[2]+2600
    
condition = (Positions[:, 1] >= ymin) & (Positions[:, 1] <= ymax) & (Positions[:, 2] >= zmin) & (Positions[:, 2] <= zmax) & (Mass > 0.1)
index = np.where(condition)[0]

del subhalo_data,Positions,Mass,condition,GalaxyPos
gc.collect()
    

#cloud.rearrange_coordinates([int(ymin+100),int(ymin+5100)],[int(zmin+100),int(zmin+5100)])

cloud=Cloud([int(ymin+100),int(ymin+5100)],[int(zmin+100),int(zmin+5100)])

    
    
    
for i in tqdm(index):
    cell_data=il.snapshot.loadSubhalo(basePath, current_snapshot_num, i, 'gas', fields=['Coordinates','Masses'])
    if len(cell_data)==1:continue

    cloud.assign_mass( cell_data['Coordinates'], cell_data['Masses'])
    del cell_data

        
del index  
gc.collect()

cloud.visualize('./areafigs/{}.png'.format(current_snapshot_num))

del cloud

gc.collect()

100%|██████████| 5000/5000 [01:03<00:00, 79.21it/s] 


25000000


100%|██████████| 2234/2234 [02:49<00:00, 13.17it/s]
/opt/conda/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:107: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)
/opt/conda/lib/python3.6/site-packages/matplotlib/colors.py:1031: RuntimeWarning: invalid value encountered in less_equal
  mask |= resdat <= 0


0

In [ ]:
for current_subfind_id,current_snapshot_num in Binary_DataIter(subfindid_snapnum_tree['SubfindID'],subfindid_snapnum_tree['SnapNum']):
    if current_snapshot_num < 40: break
    if current_snapshot_num>56:continue

    subhalo_data=il.groupcat.loadSubhalos(basePath, current_snapshot_num, fields=fields)
    Positions=subhalo_data['SubhaloPos']
    Mass=subhalo_data['SubhaloMass']
    GalaxyPos=Positions[current_subfind_id]
    
    ymin=GalaxyPos[1]-2600
    ymax=GalaxyPos[1]+2600
    
    zmin=GalaxyPos[2]-2600
    zmax=GalaxyPos[2]+2600
    
    condition = (Positions[:, 1] >= ymin) & (Positions[:, 1] <= ymax) & (Positions[:, 2] >= zmin) & (Positions[:, 2] <= zmax) & (Mass > 0.1)
    index = np.where(condition)[0]

    del subhalo_data,Positions,Mass,condition,GalaxyPos
    gc.collect()
    

#cloud.rearrange_coordinates([int(ymin+100),int(ymin+5100)],[int(zmin+100),int(zmin+5100)])

    cloud=Cloud([int(ymin+100),int(ymin+5100)],[int(zmin+100),int(zmin+5100)])

    
    
    
    for i in tqdm(index):
        cell_data=il.snapshot.loadSubhalo(basePath, current_snapshot_num, i, 'gas', fields=['Coordinates','Masses'])
        if len(cell_data)==1:continue

        cloud.assign_mass( cell_data['Coordinates'], cell_data['Masses'])
        del cell_data

        
    del index  
    gc.collect()

    cloud.visualize('./areafigs/{}.png'.format(current_snapshot_num))

    del cloud

    gc.collect()


1267